In [80]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import RobustScaler
import pickle
import math

#IMPORTS FOR MODELLING
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Activation, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras import optimizers, metrics, losses
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import tensorflow_addons as tfa;
from keras.initializers import HeNormal
from keras.regularizers import l1, l2
import os

In [81]:
#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [88]:
def encode(df, one_hot_cols, base_n_encoder_cols):
    df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True, dtype=int)
    encoder = ce.BaseNEncoder(cols=base_n_encoder_cols, return_df=True, base=2)
    df = encoder.fit_transform(df)

    return df

In [89]:
def scale(df, numeric_cols):
    scaler = RobustScaler()
    values = df[numeric_cols].values
    scaled_values = scaler.fit_transform(values)
    df.loc[:,numeric_cols] = np.array(scaled_values)

    return df

In [90]:
def encode_scale_split(df, train, validate, numeric_cols, one_hot_cols, base_n_encoder_cols):
    """encode scale and split dataset"""
    #ENCODE
    df = encode(df, one_hot_cols, base_n_encoder_cols)
    df = df.sort_values(by = ["date copy"], axis = 0, ascending = True)
    train_indice = int(train * len(df))
    validate_indice = train_indice + int(validate * len(df))
    df = df.drop(columns = ["date copy"])

    train_set = df.iloc[:train_indice]
    val_set = df.iloc[train_indice:validate_indice]
    test_set = df.iloc[validate_indice:]
    
    #PERFORM SCALING
    train_set = scale(train_set, numeric_cols)
    val_set = scale(val_set, numeric_cols)
    test_set = scale(test_set, numeric_cols)
    
    #SPLIT THE DATASET
    target_col = "AWARD_VALUE_EURO_FIN_1"
    X_train = train_set.drop(columns = [target_col]).values
    y_train = train_set[target_col].values

    X_val = val_set.drop(columns = [target_col]).values
    y_val = val_set[target_col].values

    X_test = test_set.drop(columns = [target_col]).values
    y_test = test_set[target_col].values

    return X_train, y_train, X_val, y_val, X_test, y_test

In [91]:
df = pd.read_pickle("../3.new_data/4A.df_structured_extended")
df["date copy"] = df["DT_AWARD"].copy(deep=True)

In [86]:
df.head()

,TOP_TYPE,B_ACCELERATED,TYPE_OF_CONTRACT,TAL_LOCATION_NUTS,CPV,DT_AWARD,NUMBER_OFFERS,AWARD_VALUE_EURO_FIN_1,AWARD_EST_VALUE_EURO,ISO_COUNTRY_CODE,CAE_TYPE,MAIN_ACTIVITY,B_ON_BEHALF,B_AWARDED_BY_CENTRAL_BODY,B_FRA_AGREEMENT,B_EU_FUNDS,CRIT_CODE,CRIT_PRICE_WEIGHT,date copy
0,OPE,N,W,BG315,45,2019-12-18,2.0,11473.18,12228.58,BG,1,Other,N,N,N,N,M,100.0,2019-12-18
1,OPE,N,W,BG314,31,2019-12-02,3.0,2657688.93,2671496.57,BG,3,General public\services,N,N,N,N,M,50.0,2019-12-02
2,OPE,N,W,BG422,45,2019-12-09,3.0,3058417.95,3067798.34,BG,1,Other,N,N,N,N,M,40.0,2019-12-09
3,OPE,N,U,EL434,38,2019-09-02,1.0,237356.48,242134.07,GR,8,Health,N,N,N,N,M,100.0,2019-09-02
4,OPE,N,U,PL623,48,2019-12-23,3.0,442493.81,389196.49,PL,3,General public\services,N,N,N,Y,M,60.0,2019-12-23


In [37]:
numeric_cols = ["NUMBER_OFFERS", "AWARD_EST_VALUE_EURO", "CRIT_PRICE_WEIGHT"]
base_n_encoder_cols = ["DT_AWARD", "TAL_LOCATION_NUTS"]
one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED", "ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", "B_ON_BEHALF", "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_EU_FUNDS", "CRIT_CODE", "CPV"]

X_train, y_train, X_val, y_val, X_test, y_test = encode_scale_split(df, 0.6, 0.2, one_hot_cols=one_hot_cols, numeric_cols=numeric_cols, base_n_encoder_cols=base_n_encoder_cols)

-------------------------------------------------
MODEL
-------------------------------------------------
-------------------------------------------------

In [9]:
def scheduler(epochs, lr):
    if epochs < 8:
        return lr
    else:
        return lr * math.exp(-0.1)

In [10]:
def plot_metrics(results, height, width, fig_name = None, save=True):
    plt.figure(figsize=(15, height * 5))

    axes = []
    for i in range(1, height * width + 1):
        ax = plt.subplot(height, width, i)
        axes.append(ax)

    # Find the global min and max for both mae and mse
    mae_values = np.concatenate([results[model_configuration]["history"].history["mae"] for model_configuration in results.keys()] +
                                [results[model_configuration]["history"].history["val_mae"] for model_configuration in results.keys()])
    global_min_mae, global_max_mae = np.min(mae_values), np.max(mae_values)

    mse_values = np.concatenate([results[model_configuration]["history"].history["mse"] for model_configuration in results.keys()] +
                                [results[model_configuration]["history"].history["val_mse"] for model_configuration in results.keys()])
    global_min_mse, global_max_mse = np.min(mse_values), np.max(mse_values)

    for i, model_configuration in enumerate(results.keys()):
        configuration_results = results[model_configuration]["history"].history

        for j, metric_type in enumerate(["mae", "mse"]):
            loss_train = configuration_results[metric_type]
            loss_val = configuration_results["val_" + metric_type]
            epochs = range(0, len(loss_train))

            axes[i * 2 + j].plot(epochs, loss_train, "g", label="Training {}".format(metric_type))
            axes[i * 2 + j].plot(epochs, loss_val, "b", label="Validation {}".format("val_" + metric_type))
            axes[i * 2 + j].set_title("Training and validation of model_A.{} (loss:{}, act:{}, opt:{})".format(i, 
                                                                                                             model_configuration.split("_")[0], 
                                                                                                             model_configuration.split("_")[1],
                                                                                                             model_configuration.split("_")[2]), fontsize=10)
            axes[i * 2 + j].set_xlabel("Epochs")
            axes[i * 2 + j].set_ylabel("{}".format(metric_type))

            # Set y-axis limits
            if metric_type == "mae":
                axes[i * 2 + j].set_ylim(global_min_mae, global_max_mae)
            #elif metric_type == "mse":
            #    axes[i * 2 + j].set_ylim(global_min_mse, global_max_mse)

    handles, labels = axes[0].get_legend_handles_labels()
    plt.legend(handles, labels, bbox_to_anchor=(0, 0), loc='upper right')
    plt.tight_layout()

    if save == True:
        save_path = f"../5.results/Figures/{fig_name}.png"
        plt.savefig(save_path, format='png')
    else:
        plt.show()

In [16]:
def scheduler(epochs, lr):
    if epochs < 8:
        return lr
    else:
        return lr * math.exp(-0.1)
    
def create_train_model(model_name, input_dimension, X_train, y_train, X_val, y_val, X_test, y_test, epochs,
                          initial_lr, batch_size, loss_functions, activation_functions, optimizers, save=False):
    history_models = {}

    metrics = ["mae", "mse"]

    for loss in loss_functions:
        for activation in activation_functions:
            for optimizer_name in optimizers:

                #Create a callback that saves the model's weights
                config_id = f"{model_name}_{loss}_{activation}_{optimizer_name}"
                checkpoint_path = f"../4.Models/A.structured.data.extension/{config_id}"
                cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                             monitor=loss,
                                                             save_weights_only=False,
                                                             save_best_only=True,
                                                             verbose=0,
                                                             save_freq="epoch")

                lrs_callback = tf.keras.callbacks.LearningRateScheduler(scheduler,
                                                                        verbose=0)
                seed = 10

                # Define the layers
                input_num_cat = Input(shape=input_dimension)
                x = Dense(128, activation=activation, kernel_initializer=HeNormal(seed))(input_num_cat)
                x = Dropout(rate=0.1)(x)
                x = Dense(32, activation=activation, kernel_initializer=HeNormal(seed+1))(x)
                x = Dense(4, activation=activation, kernel_initializer=HeNormal(seed+2))(x)
                regression_layer = Dense(1, activation="linear")(x)
                model_num_cat = Model(inputs=[input_num_cat],
                                      outputs=regression_layer)

                # Define the optimizer
                if optimizer_name == "adam":
                    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
                elif optimizer_name == "adagrad":
                    optimizer = tf.keras.optimizers.Adagrad(learning_rate=initial_lr)

                model_num_cat.compile(loss=loss,
                                      optimizer=optimizer,
                                      metrics=metrics)

                model_num_cat.summary()

                history = model_num_cat.fit(x=[X_train], y=y_train,
                                            validation_data=(X_val, y_val),
                                            epochs=epochs,
                                            batch_size=batch_size,
                                            callbacks=[lrs_callback,cp_callback]
                                            ,verbose=1)
                history_models[f"{loss}_{activation}_{optimizer_name}"] = history

    if save == True:
        with open(f"../5.results/A.structured_data_extension/{model_name}", "wb") as f:
            pickle.dump(history_models, f)

    return history_models

In [ ]:
#DEFINE PARAMETER GRID
loss_functions = ["mae", "mse"]
activation_functions = ["relu", "selu"]
optimizers = ["adam", "adagrad"]

results_total = create_train_model(model_name="Model_A_2", 
                      input_dimension=X_train.shape[1], 
                      X_train = X_train,
                      y_train = y_train,
                      X_val = X_val,
                      y_val = y_val,
                      X_test = X_test,
                      y_test = y_test,
                      epochs = 50,
                      initial_lr = 0.05,
                      batch_size = 16,
                      loss_functions=loss_functions,
                      activation_functions=activation_functions,
                      optimizers=optimizers)

In [ ]:
plot_metrics(results_total, height = 8, width = 2, fig_name = "Model_A", save = True)

--------------------------------------------------
EVALUATE MODELS
--------------------------------------------------
--------------------------------------------------

In [102]:
def RAE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum(abs(true_value - pred_value) for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true) for true_value in y_true)
    rae = float((delta_pred / delta_mean).item())
    return rae

In [103]:
def RMSE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum((true_value - pred_value)**2 for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true)**2 for true_value in y_true)
    rmse = float((delta_pred / delta_mean).item())
    return rmse

In [232]:
def get_test_results(y_test, X_test):
    #LOAD BEST MODELS AND MAKE PREDICTIONS
    directory = "../4.Models/A.structured.data.extension/"
    model_paths = [f"../4.Models/A.structured.data.extension/{model}" for model in os.listdir(directory)]

    model_results = {}

    for path in tqdm(model_paths):
        model = tf.keras.models.load_model(path)

        #determine metrics
        y_pred = model.predict(X_test, verbose = 0)
        mae_test = mean_absolute_error(y_test, y_pred)
        mse_test = mean_squared_error(y_test, y_pred)
        rae_test = RAE(y_test, y_pred)
        rmse_test = RMSE(y_test, y_pred)
        
        r2_metric = tf.keras.metrics.R2Score()
        r2_metric.update_state(y_test.reshape(-1, 1), y_pred)
        r2_result = r2_metric.result()
        r2_result = r2_result.numpy()
        model_results[path.split("/")[3]] = {"mae_test": mae_test, 
                          "mse_test": mse_test, 
                          "r2_results": r2_result, 
                          "rae_test": rae_test,
                          "rmse_test": rmse_test}
        
    with open("../5.results/A.structured_data_extension/test_results_all.pickle", "wb") as f:
        pickle.dump(model_results, f)
       
    return model_results

In [233]:
model_results = get_test_results(y_test, X_test)

In [234]:
model_results

{'model_A_mae_relu_adagrad': {'mae_test': 302949.5662744057,
  'mse_test': 3883555037141.471,
  'r2_results': -3.732493,
  'rae_test': 0.5221531987190247,
  'rmse_test': 4.732438564300537},
 'model_A_mae_relu_adam': {'mae_test': 186000.14183439623,
  'mse_test': 222903819599.80362,
  'r2_results': 0.7283697,
  'rae_test': 0.3205839693546295,
  'rmse_test': 0.27162933349609375},
 'model_A_mae_selu_adagrad': {'mae_test': 242023.13458274867,
  'mse_test': 1914854474087.8506,
  'r2_results': -1.3334389,
  'rae_test': 0.4171430170536041,
  'rmse_test': 2.3334124088287354},
 'model_A_mae_selu_adam': {'mae_test': 189866.03295397334,
  'mse_test': 242517776159.58865,
  'r2_results': 0.70446825,
  'rae_test': 0.327246755361557,
  'rmse_test': 0.2955305278301239},
 'model_A_mse_relu_adagrad': {'mae_test': 393499.82522411644,
  'mse_test': 1118476884400.5532,
  'r2_results': -0.3629743,
  'rae_test': 0.6782240867614746,
  'rmse_test': 1.3629757165908813},
 'model_A_mse_relu_adam': {'mae_test': 21

In [236]:
df_results = pd.DataFrame(model_results).T
df_results

,mae_test,mse_test,r2_results,rae_test,rmse_test
model_A_mae_relu_adagrad,302949.566274,3.883555e+12,-3.732493,0.522153,4.732439
model_A_mae_relu_adam,186000.141834,2.229038e+11,0.728370,0.320584,0.271629
model_A_mae_selu_adagrad,242023.134583,1.914854e+12,-1.333439,0.417143,2.333412
model_A_mae_selu_adam,189866.032954,2.425178e+11,0.704468,0.327247,0.295531
model_A_mse_relu_adagrad,393499.825224,1.118477e+12,-0.362974,0.678224,1.362976
model_A_mse_relu_adam,214160.868790,2.128029e+11,0.740679,0.369120,0.259321
model_A_mse_selu_adagrad,380808.333871,7.650686e+11,0.067689,0.656349,0.932308
model_A_mse_selu_adam,204153.649550,2.191338e+11,0.732964,0.351873,0.267036
model_B_mae_relu_adagrad,411253.094539,2.635378e+12,-2.211467,0.708822,3.211449
model_B_mae_relu_adam,190164.299764,2.336270e+11,0.715302,0.327761,0.284696


In [258]:
df_results.index = df_results.index.astype(str)
selector = "model_B"
list_indices = [index for index in df_results.index if selector in index]
df_results.loc[list_indices]

,mae_test,mse_test,r2_results,rae_test,rmse_test
model_B_mae_relu_adagrad,411253.094539,2.635378e+12,-2.211467,0.708822,3.211449
model_B_mae_relu_adam,190164.299764,2.336270e+11,0.715302,0.327761,0.284696
model_B_mae_selu_adagrad,258343.806074,4.680678e+12,-4.703865,0.445274,5.703786
model_B_mae_selu_adam,182350.073244,2.171952e+11,0.735326,0.314292,0.264672
model_B_mse_relu_adagrad,563955.954528,1.002851e+12,-0.222072,0.972016,1.222072
model_B_mse_relu_adam,385325.240309,7.237112e+11,0.118087,0.664133,0.881913
model_B_mse_selu_adagrad,328165.216537,8.258045e+11,-0.006324,0.565615,1.006322
model_B_mse_selu_adam,211905.860156,2.446373e+11,0.701885,0.365233,0.298115


In [255]:
best_results_per_model = {"model_A": {"mae_test": ["", 10.1**20],
                            "mse_test": ["", 10.1**20],
                            "r2_results":["", -10.1],
                            "rae_test": ["", 100.1],
                            "rmse_test": ["", 10.1**20]},
                "model_B": {"mae_test": ["", 10.1**20],
                            "mse_test": ["", 10.1**20],
                            "r2_results":["", -10.1],
                            "rae_test": ["", 100.1],
                            "rmse_test": ["", 10.1**20]},
                "model_C": {"mae_test": ["", 10.1**20],
                            "mse_test": ["", 10.1**20],
                            "r2_results":["", -10.1],
                            "rae_test": ["", 100.1],
                            "rmse_test": ["", 10.1**20]},
                "model_D": {"mae_test": ["", 10.1**20],
                            "mse_test": ["", 10.1**20],
                            "r2_results":["", -10.1],
                            "rae_test": ["", 100.1],
                            "rmse_test": ["", 10.1**20]}} 

best_results = {"mae_test": ["", 10.1**20],
                "mse_test": ["", 10.1**20],
                "r2_results": ["", -10.1],
                "rae_test": ["", 10.1**20],
                "rmse_test": ["", 10.1**20]}

for model in model_results.keys():
    for metric in model_results[model].keys():
        model_architecture = model[:7]
        model_configuration = model[8:]
        metric_result = model_results[model][metric]

        if metric == "r2_results":
            if best_results_per_model[model_architecture][metric][1] < metric_result:
                best_results_per_model[model_architecture][metric] = [model_configuration, metric_result]
        else:
            if best_results_per_model[model_architecture][metric][1] > metric_result:
                best_results_per_model[model_architecture][metric] = [model_configuration, metric_result]

for model in model_results.keys():
    for metric in model_results[model].keys():
        metric_result = model_results[model][metric]
        
        if metric == "r2_results":
            if best_results[metric][1] < metric_result:
                best_results[metric] = [model, metric_result]
        else:
            if best_results[metric][1] > metric_result:
                best_results[metric] = [model, metric_result]

best_results_per_model

{'model_A': {'mae_test': ['mae_relu_adam', 186000.14183439623],
  'mse_test': ['mse_relu_adam', 212802861658.51013],
  'r2_results': ['mse_relu_adam', 0.7406787],
  'rae_test': ['mae_relu_adam', 0.3205839693546295],
  'rmse_test': ['mse_relu_adam', 0.25932052731513977]},
 'model_B': {'mae_test': ['mae_selu_adam', 182350.07324409107],
  'mse_test': ['mae_selu_adam', 217195229049.92557],
  'r2_results': ['mae_selu_adam', 0.7353262],
  'rae_test': ['mae_selu_adam', 0.31429243087768555],
  'rmse_test': ['mae_selu_adam', 0.2646722197532654]},
 'model_C': {'mae_test': ['mae_relu_adam', 183480.88840212056],
  'mse_test': ['mse_selu_adam', 197436775446.40308],
  'r2_results': ['mse_selu_adam', 0.7594039],
  'rae_test': ['mae_relu_adam', 0.3162410259246826],
  'rmse_test': ['mse_selu_adam', 0.2405959814786911]},
 'model_D': {'mae_test': ['mae_selu_adam', 191557.69044261824],
  'mse_test': ['mse_relu_adam', 201889886305.2883],
  'r2_results': ['mse_relu_adam', 0.7539773],
  'rae_test': ['mae_sel

In [256]:
best_results

{'mae_test': ['model_B_mae_selu_adam', 182350.07324409107],
 'mse_test': ['model_C_mse_selu_adam', 197436775446.40308],
 'r2_results': ['model_C_mse_selu_adam', 0.7594039],
 'rae_test': ['model_B_mae_selu_adam', 0.31429243087768555],
 'rmse_test': ['model_C_mse_selu_adam', 0.2405959814786911]}

----------------------------------------------------------
LOOCV STRATEGY
----------------------------------------------------------
----------------------------------------------------------

In [92]:
def create_train_model_LOOCV(X_train, y_train, X_val, y_val, epochs,
                          initial_lr, batch_size, loss = "mae", activation = "selu"):
    
    variables = [variable for variable in df.columns if variable != "date copy" and variable != "AWARD_VALUE_EURO_FIN_1"]
    history_models = {}
    
    for variable in variables:
        numeric_cols = ["NUMBER_OFFERS", "AWARD_EST_VALUE_EURO", "CRIT_PRICE_WEIGHT"]
        base_n_encoder_cols = ["DT_AWARD", "TAL_LOCATION_NUTS"]
        one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED", "ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", 
                        "B_ON_BEHALF", "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_EU_FUNDS", "CRIT_CODE", "CPV"]
        
        if variable in numeric_cols:
            numeric_cols.remove(variable)
        elif variable in base_n_encoder_cols:
            base_n_encoder_cols.remove(variable)
        else:
            one_hot_cols.remove(variable)

        X_train, y_train, X_val, y_val, X_test, y_test = encode_scale_split(df.drop(columns = [variable]), 0.6, 0.2, one_hot_cols=one_hot_cols,
                                                                            base_n_encoder_cols=base_n_encoder_cols, numeric_cols=numeric_cols)
        
        checkpoint_path = f"../4.Models/D.LOOCV/LOOCV_{variable}"
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                                     monitor=loss,
                                                                     save_weights_only=False,
                                                                     save_best_only=True,
                                                                     verbose=0,
                                                                     save_freq="epoch")

        lrs_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

        # Define the layers
        input_num_cat = Input(shape=X_train.shape[1])
        x = Dense(128, activation=activation)(input_num_cat)
        x = Dense(32, activation=activation)(x)
        x = Dense(4, activation=activation)(x)
        regression_layer = Dense(1, activation="linear")(x)
        model_num_cat = Model(inputs=[input_num_cat],
                              outputs=regression_layer)

        model_num_cat.compile(loss=loss,
                              optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
                              metrics = ["mae", "mse", "R2Score"])

        model_num_cat.summary()

        history = model_num_cat.fit(x=[X_train], y=y_train,
                                    validation_data=(X_val, y_val),
                                    epochs=epochs,
                                    batch_size=batch_size,
                                    callbacks=[cp_callback, lrs_callback],
                                    verbose=1)
        
        history_models[f"train_test_{variable}"] = {"history": history}
        
        with open(f"../5.results/D.LOOCV/LOOCV_{variable}", "wb") as f:
            pickle.dump(history, f)

    return history

In [ ]:
history = create_train_model_LOOCV(X_train = X_train,
                                    y_train = y_train,
                                    X_val = X_val,
                                    y_val = y_val,
                                    epochs = 50,
                                    initial_lr = 0.05, 
                                    batch_size = 16)

In [104]:
def get_test_results(df):
    #LOAD BEST MODELS AND MAKE PREDICTIONS
    variables = [variable for variable in df.columns if variable != "date copy" and variable != "AWARD_VALUE_EURO_FIN_1"]
    test_results = {}

    for variable in tqdm(variables):
        numeric_cols = ["NUMBER_OFFERS", "AWARD_EST_VALUE_EURO", "CRIT_PRICE_WEIGHT"]
        base_n_encoder_cols = ["DT_AWARD", "TAL_LOCATION_NUTS"]
        one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED", "ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", 
                        "B_ON_BEHALF", "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_EU_FUNDS", "CRIT_CODE", "CPV"]
        
        if variable in numeric_cols:
            numeric_cols.remove(variable)
        elif variable in base_n_encoder_cols:
            base_n_encoder_cols.remove(variable)
        else:
            one_hot_cols.remove(variable)

        X_train, y_train, X_val, y_val, X_test, y_test = encode_scale_split(df.drop(columns = [variable]), 0.6, 0.2, one_hot_cols=one_hot_cols,
                                                                            base_n_encoder_cols=base_n_encoder_cols, numeric_cols=numeric_cols)

        path = f"../4.Models/D.LOOCV/LOOCV_{variable}"
        model = tf.keras.models.load_model(path)

        #determine metrics
        y_pred = model.predict(X_test, verbose = 0)
        mae_test = mean_absolute_error(y_test, y_pred)
        mse_test = mean_squared_error(y_test, y_pred)
        rae_test = RAE(y_test, y_pred)
        rmse_test = RMSE(y_test, y_pred)
        
        r2_metric = tf.keras.metrics.R2Score()
        r2_metric.update_state(y_test.reshape(-1, 1), y_pred)
        r2_result = r2_metric.result()
        r2_result = r2_result.numpy()
        
        test_results[variable] =  {"mae_test": mae_test, 
                          "mse_test": mse_test, 
                          "r2_results": r2_result, 
                          "rae_test": rae_test,
                          "rmse_test": rmse_test}

    with open("../5.results/D.LOOCV/test_results_all_models", "wb") as f:
        pickle.dump(test_results, f)
       
    return test_results

In [105]:
test_results = get_test_results(df)
pd.DataFrame(test_results).transpose().to_csv("../5.results/D.LOOCV/test_results.csv")

100%|██████████| 17/17 [00:19<00:00,  1.12s/it]


In [119]:
df_test_results = pd.DataFrame(test_results).transpose()
df_test_results

,mae_test,mse_test,r2_results,rae_test,rmse_test
TOP_TYPE,170044.461098,1.808528e+11,0.766209,0.304064,0.233790
B_ACCELERATED,169143.006966,1.739573e+11,0.775123,0.302452,0.224876
TYPE_OF_CONTRACT,173973.993814,1.892193e+11,0.755394,0.311091,0.244605
TAL_LOCATION_NUTS,168060.610326,1.783123e+11,0.769493,0.300517,0.230506
CPV,176894.731513,1.927943e+11,0.750772,0.316314,0.249227
DT_AWARD,172209.013605,1.721983e+11,0.777397,0.307934,0.222602
NUMBER_OFFERS,178872.841337,1.906428e+11,0.753554,0.319851,0.246445
AWARD_EST_VALUE_EURO,485760.894911,8.271894e+11,-0.069319,0.868614,1.069318
ISO_COUNTRY_CODE,175581.135234,2.064894e+11,0.733068,0.313964,0.266930
CAE_TYPE,169614.855765,1.782727e+11,0.769544,0.303296,0.230454


In [120]:
def closest_to_percentile(arr, percentile):
    target_value = np.percentile(arr, percentile)
    closest_value = min(arr, key=lambda x: abs(x - target_value))

    return closest_value

8


In [125]:
results_mae = closest_to_percentile(df_test_results["mae_test"], 50)
results_mse = closest_to_percentile(df_test_results["mse_test"], 50)
results_r2 = closest_to_percentile(df_test_results["r2_results"], 50)
results_rae = closest_to_percentile(df_test_results["rae_test"], 50)
results_rmse = closest_to_percentile(df_test_results["rmse_test"], 50)

print("results_mae:", results_mae, "\n\n",
      "results_mse:", results_mse, "\n\n",
      "results_r2:", results_r2, "\n\n",
      "results_rae:", results_rae, "\n\n",
      "results_rmse:", results_rmse, "\n\n")

results_mae: 173606.20790312425 

 results_mse: 189219322965.03958 

 results_r2: 0.7553936243057251 

 results_rae: 0.31043198704719543 

 results_rmse: 0.2446054071187973 




In [152]:
def rank_features_by_percentile(df):
    metrics = df.columns
    ranks = pd.DataFrame(index=df.index, columns=metrics)

    for metric in metrics:
        target_value = np.percentile(df[metric], 50)
        ranks[metric] = ((df[metric] - target_value) / target_value) * 100

    ranks["r2_results"] = ranks["r2_results"] * -1
    ranks['Overall_Rank'] = ranks.sum(axis=1)
    ranks = ranks.sort_values(by='Overall_Rank', ascending=False)

    return ranks

In [153]:
result_ranks = rank_features_by_percentile(df_test_results)
result_ranks

,mae_test,mse_test,r2_results,rae_test,rmse_test,Overall_Rank
AWARD_EST_VALUE_EURO,179.806178,337.159051,109.176526,179.808156,337.160530,1143.110440
ISO_COUNTRY_CODE,1.137590,9.127025,2.955450,1.137844,9.126942,23.484850
B_FRA_AGREEMENT,1.454052,5.204147,1.685167,1.454413,5.204295,15.002074
MAIN_ACTIVITY,0.000000,3.777153,1.223089,0.000000,3.776995,8.777238
CPV,1.894243,1.889321,0.611785,1.894672,1.889299,8.179320
NUMBER_OFFERS,3.033667,0.752274,0.243597,3.034109,0.752218,7.815864
CRIT_PRICE_WEIGHT,0.712118,1.635355,0.529550,0.712494,1.635418,5.224935
CRIT_CODE,0.322086,0.888321,0.287650,0.322598,0.888220,2.708874
TYPE_OF_CONTRACT,0.211851,0.000000,-0.000000,0.212339,0.000000,0.424190
B_EU_FUNDS,-0.966040,-0.782594,-0.253421,-0.965365,-0.782604,-3.750024


In [155]:
result_ranks.to_csv("../5.results/D.LOOCV/test_result_ranking.csv")